In [1]:
import pandas as pd
import gensim.models.keyedvectors as word2vec
from numpy.linalg import norm
import numpy as np
from scipy.spatial.distance import cosine
from sklearn.decomposition import PCA
from sklearn.manifold import MDS
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
def cos_sim(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b)

In [3]:
df = pd.read_csv("./word2vec.csv",header=0, keep_default_na=False)
#df = pd.read_csv("./word2vec_debiased.csv",header=0, keep_default_na=False)
model =  word2vec.KeyedVectors.load_word2vec_format('./word2vec_50k.bin', binary=True)

In [4]:
df.head()

,word
0,prison
1,agony
2,vomit
3,rotten
4,cancer


In [5]:
list(model.vocab.keys())

['outparcels',
 'nunnery',
 'telmisartan',
 'southbridge',
 'ampitheatre',
 'vani',
 'circuitry',
 'pantheistic',
 'hanging',
 'bromelain',
 'woody',
 'trawling',
 'hastily',
 'spidery',
 'sation',
 'disobeying',
 'caner',
 'coarctation',
 'nonfactor',
 'turbodiesel',
 'gaa',
 'crossbar',
 'symbologist',
 'rumbustious',
 'midgame',
 'cyclodextrin',
 'hahahah',
 'wracked',
 'pigment',
 'capoeira',
 'rawhide',
 'hydrolyzate',
 'screaming',
 'cyberthreats',
 'chesed',
 'grueling',
 'broiler',
 'wooden',
 'wholemeal',
 'penghulus',
 'wednesday',
 'broiled',
 'crotch',
 'highveld',
 'guardsmen',
 'sooty',
 'lossmaking',
 'accountholders',
 'scraper',
 'mountainbike',
 'rosemaling',
 'bannister',
 'reengineering',
 'kodak',
 'targa',
 'prosody',
 'inanimate',
 'dormancy',
 'semicircular',
 'cooking',
 'salsify',
 'warmongering',
 'usenet',
 'databank',
 'schoolfriends',
 'evolutionism',
 'moksha',
 'wheatgrass',
 'wildchild',
 'brainwashed',
 'affiliates',
 'chine',
 'ching',
 'perfunctorily

In [6]:
# calculate bias direction when we have group of words not pairs
def groupBiasDirection(gp1, gp2):
    #print(gp1,gp2)
    dim = len(model["he"])
    g1,g2 = np.zeros((dim,), dtype=float), np.zeros((dim,), dtype=float)
    for p in gp1:
        p = p.strip()
        if p not in model:
            continue
        p_vec = model[p]/norm(model[p])
        g1 = np.add(g1,p_vec)

    for q in gp2:
        q = q.strip()
        if q not in model:
            continue
        q_vec = model[q]/norm(model[q])
        g2 = np.add(g2,q_vec) 

    g1, g2 = g1/norm(g1), g2/norm(g2)
    return (g1,g2)

In [7]:
gender_bias = ["man,boy,he,father,son,guy,male,his,himself,john".split(","),
               "woman,girl,she,mother,daughter,gal,female,her,herself,mary".split(",")]
#eco_bias = [("rich","wealthy"),("poor","impoverished")]
race_bias = ["aisha,keisha,tamika,lakisha,tanisha,latoya,kenya,latonya,ebony,rasheed,tremayne,kareem,darnell,tyrone,hakim,jamal,leroy,jermaine".split(","),
             "emily,anne,jill,allison,laurie,sarah,meredith,carrie,kristen,todd,neil,geoffrey,brett,brendan,greg,matthew,jay,brad".split(",")]
religion_bias = ["baptism, messiah, catholicism, resurrection, christianity, salvation, protestant, gospel, trinity, jesus, christ, christian, cross, catholic, church".split(","),
                "allah, ramadan, turban, emir, salaam, sunni, koran, imam, sultan, prophet, veil, ayatollah, shiite, mosque, islam, sheik, muslim, muhammad".split(",")]
sentiment_bias = ["caress, freedom, health, love, peace, cheer, friend, heaven, loyal, pleasure, diamond, gentle, honest, lucky, rainbow, diploma, gift, honor, miracle, sunrise, family, happy, laughter, paradise, vacation".split(","),
                 "abuse, crash, filth, murder, sickness, accident, death, grief, poison, stink, assault, disaster, hatred, pollute, tragedy, divorce, jail, poverty, ugly, cancer, kill, rotten, vomit, agony, prison".split(",")]
age_bias = ["tiffany,michelle,cindy,kristy,brad,eric,joey,billy".split(","),
           "ethel,bernice,gertrude,agnes,cecil,wilbert,mortimer,edgar".split(",")]
bias_words = {"gender":gender_bias, "religion":religion_bias, "race":race_bias, "age":age_bias, "sentiment":sentiment_bias}

In [8]:
bias_words

{'gender': [['man',
   'boy',
   'he',
   'father',
   'son',
   'guy',
   'male',
   'his',
   'himself',
   'john'],
  ['woman',
   'girl',
   'she',
   'mother',
   'daughter',
   'gal',
   'female',
   'her',
   'herself',
   'mary']],
 'religion': [['baptism',
   ' messiah',
   ' catholicism',
   ' resurrection',
   ' christianity',
   ' salvation',
   ' protestant',
   ' gospel',
   ' trinity',
   ' jesus',
   ' christ',
   ' christian',
   ' cross',
   ' catholic',
   ' church'],
  ['allah',
   ' ramadan',
   ' turban',
   ' emir',
   ' salaam',
   ' sunni',
   ' koran',
   ' imam',
   ' sultan',
   ' prophet',
   ' veil',
   ' ayatollah',
   ' shiite',
   ' mosque',
   ' islam',
   ' sheik',
   ' muslim',
   ' muhammad']],
 'race': [['aisha',
   'keisha',
   'tamika',
   'lakisha',
   'tanisha',
   'latoya',
   'kenya',
   'latonya',
   'ebony',
   'rasheed',
   'tremayne',
   'kareem',
   'darnell',
   'tyrone',
   'hakim',
   'jamal',
   'leroy',
   'jermaine'],
  ['emily',
 

In [9]:
all_words = list(model.vocab.keys()) #[:10000]
#all_words = df["word"].tolist()
df = pd.DataFrame({"word":all_words})
for bias_type in bias_words:
    bias_w = bias_words[bias_type]
    df[bias_type] = None
    g1, g2 = groupBiasDirection(bias_w[0], bias_w[1])
    for index, row in df.iterrows():
        w = row["word"]
        # assuming group bias "Quantification algo"
        df.at[index, bias_type] = round(cosine(g1,model[w])-cosine(g2,model[w]),4)

In [10]:
gen_max, gen_min = df["gender"].max(), df["gender"].min()
sen_max, sen_min = df["sentiment"].max(), df["sentiment"].min()
race_max, race_min = df["race"].max(), df["race"].min()
relg_max, relg_min = df["religion"].max(), df["religion"].min()
age_max, age_min = df["age"].max(), df["age"].min()

print("Gender: ",gen_min,gen_max)
print("Sentiment: ",sen_min, sen_max)
print("Race: ",race_min, race_max)
print("Religion: ",relg_min, relg_max)
print("Age: ",age_min, age_max)

Gender:  -0.329 0.344
Sentiment:  -0.3683 0.3651
Race:  -0.3612 0.2574
Religion:  -0.3848 0.4055
Age:  -0.3053 0.4322


In [11]:
# normalization of bias scores
for index, row in df.iterrows():
    if row["gender"]>0:
        df.at[index, "gender"] = row["gender"]/gen_max
    else:
        df.at[index, "gender"] = -1*row["gender"]/gen_min
        
    if row["race"]>0:
        df.at[index, "race"] = row["race"]/race_max
    else:
        df.at[index, "race"] = -1*row["race"]/race_min
    
    if row["sentiment"]>0:
        df.at[index, "sentiment"] = row["sentiment"]/sen_max
    else:
        df.at[index, "sentiment"] = -1*row["sentiment"]/sen_min
        
    if row["religion"]>0:
        df.at[index, "religion"] = row["religion"]/relg_max
    else:
        df.at[index, "religion"] = -1*row["religion"]/relg_min
    
    if row["age"]>0:
        df.at[index, "age"] = row["age"]/age_max
    else:
        df.at[index, "age"] = -1*row["age"]/age_min  

In [12]:
df.head()

,word,gender,religion,race,age,sentiment
0,outparcels,0.0159884,0.174353,0.011655,0.185562,0.111202
1,nunnery,0.328779,-0.196206,-0.0251938,-0.173272,-0.00570187
2,telmisartan,0.0697674,0.160049,-0.104374,0.0402591,0.080252
3,southbridge,-0.00182371,0.0631319,-0.00442968,0.0326238,-0.0955743
4,ampitheatre,0.0906977,-0.194127,-0.0722591,-0.00229283,-0.342927


In [13]:
gen_max, gen_min = df["gender"].max(), df["gender"].min()
sen_max, sen_min = df["sentiment"].max(), df["sentiment"].min()
race_max, race_min = df["race"].max(), df["race"].min()
relg_max, relg_min = df["religion"].max(), df["religion"].min()
age_max, age_min = df["age"].max(), df["age"].min()

print("Gender: ",gen_min,gen_max)
print("Sentiment: ",sen_min, sen_max)
print("Race: ",race_min, race_max)
print("Religion: ",relg_min, relg_max)
print("Age: ",age_min, age_max)

Gender:  -1.0 1.0
Sentiment:  -1.0 1.0
Race:  -1.0 1.0
Religion:  -1.0 1.0
Age:  -1.0 1.0


In [14]:
'''
# calculaue principal component
matrix = []
for w in df["word"].tolist():
    matrix.append(model[w])
matrix = np.array(matrix)

dis_mat = cosine_similarity(matrix)
dis_mat.shape

embd = MDS(n_components=2, dissimilarity="precomputed").fit_transform(dis_mat)
df["scatter_x"] = embd[:,0]
df["scatter_y"] = embd[:,1]

df["semantic"] = MDS(n_components=1, dissimilarity="precomputed").fit_transform(dis_mat).flatten()
'''

'\n# calculaue principal component\nmatrix = []\nfor w in df["word"].tolist():\n    matrix.append(model[w])\nmatrix = np.array(matrix)\n\ndis_mat = cosine_similarity(matrix)\ndis_mat.shape\n\nembd = MDS(n_components=2, dissimilarity="precomputed").fit_transform(dis_mat)\ndf["scatter_x"] = embd[:,0]\ndf["scatter_y"] = embd[:,1]\n\ndf["semantic"] = MDS(n_components=1, dissimilarity="precomputed").fit_transform(dis_mat).flatten()\n'

In [15]:
df.head()

,word,gender,religion,race,age,sentiment
0,outparcels,0.0159884,0.174353,0.011655,0.185562,0.111202
1,nunnery,0.328779,-0.196206,-0.0251938,-0.173272,-0.00570187
2,telmisartan,0.0697674,0.160049,-0.104374,0.0402591,0.080252
3,southbridge,-0.00182371,0.0631319,-0.00442968,0.0326238,-0.0955743
4,ampitheatre,0.0906977,-0.194127,-0.0722591,-0.00229283,-0.342927


In [16]:
df.describe()

,word,gender,religion,race,age,sentiment
count,50000,50000.000000,50000.000000,50000.000000,50000.000000,50000.00000
unique,50000,3760.000000,3770.000000,3466.000000,3835.000000,4616.00000
top,backbreaking,0.053488,-0.018451,0.008936,-0.026531,-0.01792
freq,1,57.000000,54.000000,53.000000,45.000000,43.00000


In [17]:
df.shape

(50000, 6)

In [19]:
df.to_csv("../data/all_biases_50k.csv", encoding='utf-8', index=False)

In [11]:
'''
words = df["word"].tolist()
for index, row in df.iterrows():
    w = row["word"]
    df.at[index, "gender"] = round(cosine(g1,model[w])-cosine(g2,model[w]),4)
    df.at[index, "eco"] = round(cosine(g3,model[w])-cosine(g4,model[w]),4)
    df.at[index, "race"] = round(cosine(g5,model[w])-cosine(g6,model[w]),4)
    df.at[index, "semantic"] = round(cos_sim(g,model[w]),5) 
'''

In [13]:
df = df.sort_values('semantic')
df = df[['word','gender','eco','race']]

In [39]:
thresh = 0.15

In [41]:
# filtering: if either of biases is greater than thresh
df_final = df[(abs(df["gender"]) > thresh) | (abs(df["eco"]) > thresh) | (abs(df["race"]) > thresh)]

In [42]:
df_final.shape

(7560, 4)

In [47]:
# filtering: if total bias sum is less than thresh
sum_thresh = 0.45
df_final = df[ (abs(df["gender"])+abs(df["eco"])+abs(df["race"]))> sum_thresh]

In [48]:
df_final.shape

(3864, 4)

In [49]:
df_final.to_csv("../data/mutliple_biases_norm.csv", encoding='utf-8', index=False)

## Miscellaneous

In [50]:
df.loc[df["word"]=="good"]

,word,gender,eco,race
411,good,-0.321548,0.283569,-0.105553


In [51]:
df_final.loc[df_final["word"]=="good"]

,word,gender,eco,race
411,good,-0.321548,0.283569,-0.105553
